In [2]:
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install xgboost


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

%cd data/Mens/Season/
ordinal_df = pd.read_csv('2024/MMasseyOrdinals_2024.csv')
games_df = pd.read_csv('2024/MRegularSeasonDetailedResults_2024.csv')


c:\Users\Dowee\OneDrive\Desktop\School\Intelligent Systems\Project 1\March-Madness-Predictor\data\Mens\Season


C:\Users\Dowee\AppData\Roaming\Python\Python311\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
ordinal_df


,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2024,9,AP,1104,22
1,2024,9,AP,1112,3
2,2024,9,AP,1116,14
3,2024,9,AP,1124,15
4,2024,9,AP,1160,25
...,...,...,...,...,...
259479,2024,114,WLK,1474,273
259480,2024,114,WLK,1475,326
259481,2024,114,WLK,1476,356
259482,2024,114,WLK,1477,319


In [5]:
games_df['Week'] = ((games_df['DayNum']-1)/7 +1)
games_df['Week'] = games_df['Week'].apply(np.floor)


ordinal_df['Week'] = ((ordinal_df['RankingDayNum']-1)/7 +1)
ordinal_df['Week'] = ordinal_df['Week'].apply(np.floor)
games_df


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Week
0,2024,0,1101,64,1329,59,A,0,26,57,...,8,20,6,26,13,12,9,2,16,0.0
1,2024,0,1103,81,1355,75,A,0,26,57,...,11,13,5,18,14,12,8,2,17,0.0
2,2024,0,1104,105,1287,73,H,0,32,57,...,12,19,11,15,7,14,6,3,25,0.0
3,2024,0,1112,122,1288,59,H,0,42,76,...,8,15,6,17,10,25,3,6,25,0.0
4,2024,0,1114,71,1402,66,H,0,22,59,...,13,22,17,31,14,22,2,4,23,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4865,2024,114,1454,75,1237,70,A,0,25,57,...,16,23,7,19,15,13,7,2,21,17.0
4866,2024,114,1455,74,1412,66,A,0,27,54,...,15,22,9,21,15,12,8,3,12,17.0
4867,2024,114,1459,91,1359,69,H,0,32,59,...,20,28,10,18,10,14,6,0,19,17.0
4868,2024,114,1462,91,1177,58,H,0,35,67,...,11,14,5,25,11,18,6,4,12,17.0


In [6]:
def calculate_additional_stats(df):
    """
    Adds calculated statistics for two-point field goals to the DataFrame.

    Parameters:
    - df (DataFrame): The original game results DataFrame.

    Returns:
    - DataFrame: The modified DataFrame with additional stats.
    """
    df['WFGM2'] = df['WFGM'] - df['WFGM3']
    df['WFGA2'] = df['WFGA'] - df['WFGA3']
    df['LFGM2'] = df['LFGM'] - df['LFGM3']
    df['LFGA2'] = df['LFGA'] - df['LFGA3']
    return df

In [7]:
def prepare_team_stats(df):
    """
    Prepares and aggregates team statistics and statistics against from game results.

    Parameters:
    - df (DataFrame): The game results DataFrame with additional stats.

    Returns:
    - DataFrame: A DataFrame with average stats per team and stats against.
    """
    df = calculate_additional_stats(df)
    # Stats when the team wins
    win_stats = df[['WTeamID','DayNum','Week', 'WFGM', 'WFGA', 'WFGM2', 'WFGA2', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']].copy()
    win_stats.columns = ['TeamID','DayNum','Week', 'FGM', 'FGA', 'FGM2', 'FGA2', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
    
    # Stats against the team when it wins (opponents' performance)
    win_against_stats = df[['WTeamID','DayNum','Week', 'LFGM', 'LFGA', 'LFGM2', 'LFGA2', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']].copy()
    win_against_stats.columns = ['TeamID','DayNum','Week', 'FGMA', 'FGAA', 'FGM2A', 'FGA2A', 'FGM3A', 'FGA3A', 'FTMA', 'FTAA', 'ORA', 'DRA', 'AstA', 'TOA', 'StlA', 'BlkA', 'PFA']

    # Stats when the team loses
    lose_stats = df[['LTeamID','DayNum','Week', 'LFGM', 'LFGA', 'LFGM2', 'LFGA2', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']].copy()
    lose_stats.columns = ['TeamID', 'DayNum','Week','FGM', 'FGA', 'FGM2', 'FGA2', 'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF']
    
    # Stats against the team when it loses (opponents' performance)
    lose_against_stats = df[['LTeamID','DayNum','Week','WFGM', 'WFGA', 'WFGM2', 'WFGA2', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']].copy()
    lose_against_stats.columns = ['TeamID','DayNum','Week', 'FGMA', 'FGAA', 'FGM2A', 'FGA2A', 'FGM3A', 'FGA3A', 'FTMA', 'FTAA', 'ORA', 'DRA', 'AstA', 'TOA', 'StlA', 'BlkA', 'PFA']

    # Combine winning and losing stats
    all_stats = pd.concat([win_stats, lose_stats]).sort_values(by=['TeamID', 'DayNum'])
    all_against_stats = pd.concat([win_against_stats, lose_against_stats]).sort_values(by=['TeamID', 'DayNum'])

    Week_DayNum_for =  all_stats[['TeamID', 'DayNum', 'Week']].reset_index(drop=True)
    Week_DayNum_against = all_against_stats[['TeamID', 'DayNum', 'Week']].reset_index(drop=True)

    all_stats.drop(columns=['Week', 'DayNum'], inplace=True)
    all_against_stats.drop(columns=['Week', 'DayNum'], inplace=True)

    stats_rolling = all_stats.groupby('TeamID').rolling(window=10, min_periods=1).mean().reset_index(drop=True)
    against_rolling = all_against_stats.groupby('TeamID').rolling(window=10, min_periods=1).mean().reset_index(drop=True)

    stats_rolling = pd.concat([Week_DayNum_for, stats_rolling], axis=1)
    # print(stats_rolling.columns)
    # print(stats_rolling.head(29))
    against_rolling = pd.concat([Week_DayNum_against, against_rolling], axis=1)

    # # Calculate the mean for stats and stats against separately
    # avg_stats = all_stats.groupby('TeamID').mean().reset_index()
    # avg_against_stats = all_against_stats.groupby('TeamID').mean().reset_index()

    merged_stats = pd.merge(stats_rolling, against_rolling, on=['TeamID','DayNum','Week'], suffixes=('', '_A'))
    return merged_stats

In [8]:


teams_stats_weekly_df = prepare_team_stats(games_df)
# ordinal_df = ordinal_df.sort_values(by=['TeamID', 'RankingDayNum']).reset_index(drop=True)
# ordinal_df = ordinal_df.rename(columns={'RankingDayNum':'DayNum'})

# system_names = ordinal_df['SystemName'].unique()
# teams_names = ordinal_df['TeamID'].unique()
# system_no_rank_all_teams = []

# for system in system_names:
#     teams_in_system = ordinal_df[ordinal_df['SystemName'] == system]['TeamID'].unique()
#     if len(teams_in_system) != len(teams_names):
#         system_no_rank_all_teams.append(system)

# ordinal_df = ordinal_df[~ordinal_df['SystemName'].isin(system_no_rank_all_teams)]
# ordinal_pivot = ordinal_df.pivot_table(index=['TeamID', 'DayNum', 'Week'], columns='SystemName', values='OrdinalRank').reset_index()
# ordinal_pivot.sort_values(by=['TeamID', 'DayNum'])
# ordinal_pivot = ordinal_pivot.ffill()
# ordinal_pivot = ordinal_pivot.groupby('TeamID').apply(lambda x: x.interpolate(method='linear', limit_direction='both')).reset_index(drop=True)

ordinal_df = ordinal_df.sort_values(by=['TeamID']).reset_index(drop=True)
ordinal_df = ordinal_df.drop(columns=['RankingDayNum'])

system_names = ordinal_df['SystemName'].unique()
teams_names = ordinal_df['TeamID'].unique()
system_no_rank_all_teams = []

for system in system_names:
    teams_in_system = ordinal_df[ordinal_df['SystemName'] == system]['TeamID'].unique()
    if len(teams_in_system) != len(teams_names):
        system_no_rank_all_teams.append(system)

ordinal_df = ordinal_df[~ordinal_df['SystemName'].isin(system_no_rank_all_teams)]
ordinal_pivot = ordinal_df.pivot_table(index=['TeamID'], columns='SystemName', values='OrdinalRank').reset_index()
ordinal_pivot.sort_values(by=['TeamID'])
ordinal_pivot = ordinal_pivot.groupby('TeamID').mean().reset_index()


ordinal_pivot.columns

# ordinal_pivot[(ordinal_pivot['TeamID'] == 1104)]


Index(['TeamID', '7OT', 'ARG', 'BAR', 'BBT', 'BIH', 'BMN', 'BNZ', 'BWE', 'COL',
       'DCI', 'DII', 'DOK', 'DOL', 'DP', 'DUN', 'EBP', 'EMK', 'ESR', 'FAS',
       'HAS', 'INC', 'JJK', 'JNG', 'KPK', 'LMC', 'LOG', 'MAS', 'MB', 'MGS',
       'MMG', 'MOR', 'NET', 'NOL', 'OMY', 'PAC', 'PGH', 'PIR', 'POM', 'REW',
       'RMS', 'RPI', 'RT', 'RTH', 'RTP', 'SEL', 'SFX', 'SMS', 'SPR', 'SRS',
       'STH', 'STY', 'TPR', 'TRK', 'TRP', 'WIL', 'WLK', 'WOL'],
      dtype='object', name='SystemName')

In [9]:
games_df[(games_df['WTeamID'] == 1104)]

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LAst,LTO,LStl,LBlk,LPF,Week,WFGM2,WFGA2,LFGM2,LFGA2
2,2024,0,1104,105,1287,73,H,0,32,57,...,7,14,6,3,25,0.0,22,34,20,39
171,2024,4,1104,102,1232,80,H,0,33,64,...,10,13,8,2,24,1.0,20,41,17,27
345,2024,8,1104,102,1375,46,H,0,34,55,...,6,11,6,2,21,2.0,22,32,9,31
466,2024,11,1104,98,1273,67,H,0,35,59,...,7,8,6,3,24,2.0,27,40,23,50
894,2024,19,1104,99,1332,91,N,0,32,64,...,20,12,9,5,24,3.0,23,40,29,49
1296,2024,28,1104,89,1117,65,H,0,28,64,...,10,14,5,7,19,4.0,15,33,16,35
1949,2024,47,1104,111,1184,67,H,0,40,69,...,8,13,7,5,17,7.0,21,31,16,42
2037,2024,54,1104,101,1251,56,N,0,35,60,...,15,9,2,1,17,8.0,21,32,11,28
2287,2024,61,1104,78,1435,75,A,0,26,53,...,6,8,10,3,17,9.0,14,23,19,39
2463,2024,64,1104,74,1376,47,H,0,26,54,...,8,16,8,2,13,10.0,11,20,12,30


In [10]:
"""
This code performs the following operations:

1. Sorts the 'ordinal_pivot' DataFrame by 'TeamID' and 'DayNum' columns.
2. Sorts the 'teams_stats_weekly_df' DataFrame by 'TeamID' and 'DayNum' columns.
3. Merges the sorted 'teams_stats_weekly_df' and 'ordinal_pivot' DataFrames using 'DayNum' and 'TeamID' columns in a backward direction.
4. Sorts the resulting DataFrame by 'TeamID' and 'DayNum' columns and resets the index.
5. Extracts the columns starting from the 35th column and assigns them to 'rank_columns' variable.
6. Fills the missing values in 'rank_columns' for each 'TeamID' using backward filling.
7. Filters the resulting DataFrame to include only rows where 'TeamID' is equal to 1104.

Parameters:
    - ordinal_pivot: DataFrame containing ordinal rankings.
    - teams_stats_weekly_df: DataFrame containing weekly team statistics.

Returns:
    - weekly_stats_w_rating: DataFrame with sorted and merged data, filled with missing values, and filtered by 'TeamID' 1104.
"""
ordinal_pivot = ordinal_pivot.sort_values(by=['TeamID'])

teams_stats_weekly_df = teams_stats_weekly_df.sort_values(by=['TeamID'])
weekly_stats_w_rating = pd.merge(teams_stats_weekly_df, ordinal_pivot, on='TeamID', suffixes=('', '_A'))
weekly_stats_w_rating = weekly_stats_w_rating.sort_values(by=['TeamID']).reset_index(drop=True)
rank_columns = weekly_stats_w_rating.columns[34:]

weekly_stats_w_rating[rank_columns] = weekly_stats_w_rating.groupby('TeamID')[rank_columns].bfill()
weekly_stats_w_rating = weekly_stats_w_rating.dropna(axis = 1, how= 'any')
weekly_stats_w_rating



,TeamID,DayNum,Week,FGM,FGA,FGM2,FGA2,FGM3,FGA3,FTM,...,SPR,SRS,STH,STY,TPR,TRK,TRP,WIL,WLK,WOL
0,1101,0,0.0,26.000000,57.000000,21.000000,39.000000,5.000000,18.000000,7.000000,...,230.785714,220.750000,225.416667,197.9375,222.9,188.0625,201.0000,215.1875,232.166667,251.0
1,1101,23,4.0,23.714286,57.428571,18.000000,41.142857,5.714286,16.285714,15.142857,...,230.785714,220.750000,225.416667,197.9375,222.9,188.0625,201.0000,215.1875,232.166667,251.0
2,1101,45,7.0,23.600000,57.400000,18.200000,41.300000,5.400000,16.100000,19.900000,...,230.785714,220.750000,225.416667,197.9375,222.9,188.0625,201.0000,215.1875,232.166667,251.0
3,1101,41,6.0,23.400000,55.900000,17.900000,39.600000,5.500000,16.300000,19.300000,...,230.785714,220.750000,225.416667,197.9375,222.9,188.0625,201.0000,215.1875,232.166667,251.0
4,1101,30,5.0,23.333333,57.444444,17.888889,40.777778,5.444444,16.666667,17.666667,...,230.785714,220.750000,225.416667,197.9375,222.9,188.0625,201.0000,215.1875,232.166667,251.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9735,1478,76,11.0,25.600000,59.000000,15.100000,31.300000,10.500000,27.700000,9.800000,...,307.071429,322.833333,331.916667,317.0000,314.9,334.8750,328.3125,327.8750,309.916667,293.0
9736,1478,74,11.0,24.600000,59.500000,15.400000,32.700000,9.200000,26.800000,10.800000,...,307.071429,322.833333,331.916667,317.0000,314.9,334.8750,328.3125,327.8750,309.916667,293.0
9737,1478,61,9.0,24.500000,59.000000,15.300000,31.500000,9.200000,27.500000,12.300000,...,307.071429,322.833333,331.916667,317.0000,314.9,334.8750,328.3125,327.8750,309.916667,293.0
9738,1478,29,5.0,22.714286,59.857143,14.428571,32.000000,8.285714,27.857143,11.857143,...,307.071429,322.833333,331.916667,317.0000,314.9,334.8750,328.3125,327.8750,309.916667,293.0


In [11]:
def prepare_matchup_data(games_df, stats):
    """
    Merges game data with team stats to prepare matchup data.

    Parameters:
    - games_df (DataFrame): The DataFrame containing game results.
    - avg_stats (DataFrame): The DataFrame containing average stats per team.

    Returns:
    - DataFrame: Matchup data with team stats and game outcome.
    """
    stats.set_index('TeamID', inplace=True)
    processed_data = []

    for _, row in games_df.iterrows():
        team_1, team_2 = sorted((row['WTeamID'], row['LTeamID']))
        team_1_won = 1 if team_1 == row['WTeamID'] else 0
        team_1_stats = stats.loc[team_1].add_prefix('team_1_').iloc[-1]
        team_2_stats = stats.loc[team_2].add_prefix('team_2_').iloc[-1]
        
        matchup_data = {
            'Season': row['Season'],
            'DayNum': row['DayNum'],
            'team_1': team_1,
            'team_2': team_2,
            'team_1_won': team_1_won
        }
        matchup_data.update(team_1_stats)
        matchup_data.update(team_2_stats)

        processed_data.append(matchup_data)

    return pd.DataFrame(processed_data)

In [12]:
test = prepare_matchup_data(games_df, weekly_stats_w_rating)

In [13]:
test.isna().sum().drop_duplicates()
test

,Season,DayNum,team_1,team_2,team_1_won,team_1_DayNum,team_1_Week,team_1_FGM,team_1_FGA,team_1_FGM2,...,team_2_SPR,team_2_SRS,team_2_STH,team_2_STY,team_2_TPR,team_2_TRK,team_2_TRP,team_2_WIL,team_2_WLK,team_2_WOL
0,2024,0,1101,1329,1,94.0,14.0,25.7,61.1,20.5,...,139.928571,95.166667,113.000000,57.6875,122.5,91.6250,92.1875,117.8750,117.916667,234.0
1,2024,0,1103,1355,1,103.0,15.0,25.5,57.9,18.3,...,179.142857,160.333333,175.583333,178.6250,171.4,150.1875,162.4375,184.6250,176.666667,218.0
2,2024,0,1104,1287,1,93.0,14.0,28.4,61.1,17.3,...,115.571429,155.166667,108.750000,216.1875,171.6,164.3125,156.3750,139.9375,129.083333,81.0
3,2024,0,1112,1288,1,87.0,13.0,31.0,63.7,23.3,...,340.785714,346.500000,332.500000,342.4375,348.0,339.1250,344.8125,335.5000,332.833333,347.0
4,2024,0,1114,1402,1,110.0,16.0,28.2,56.5,23.1,...,220.928571,234.500000,216.583333,232.8125,247.8,239.6250,204.1875,237.1875,210.166667,140.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4865,2024,114,1237,1454,0,96.0,14.0,24.7,56.0,21.5,...,272.214286,266.833333,290.333333,259.8750,241.8,247.7500,254.2500,223.1875,276.333333,242.0
4866,2024,114,1412,1455,0,97.0,14.0,27.0,59.7,21.1,...,143.500000,122.000000,143.666667,129.9375,144.0,119.6875,119.1875,117.0000,141.416667,133.0
4867,2024,114,1359,1459,0,114.0,17.0,28.1,56.4,18.6,...,201.142857,200.916667,234.083333,199.5625,203.3,238.5000,225.3750,203.0000,208.916667,190.0
4868,2024,114,1177,1462,0,67.0,10.0,22.1,51.6,16.0,...,60.857143,42.166667,50.416667,38.6250,41.6,50.1250,41.3125,54.5625,47.833333,131.0


In [14]:

def fit_model_scalar(model_param, model_name, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3270)

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Fit the model
    model_param.fit(X_train_scaled, y_train)
    y_pred = model_param.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name} scalar accuracy: {accuracy:.2f}')

In [15]:
def fit_model(model_param, model_name, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3270)

    # sfs = SequentialFeatureSelector(model_param, n_features_to_select=10)
    # sfs.fit(X_train, y_train)
    # X_train = sfs.transform(X_train)
    # X_test = sfs.transform(X_test)
    model_param.fit(X_train, y_train)
    y_pred = model_param.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name} accuracy: {accuracy:.2f}')

In [16]:
# matchups = prepare_matchup_data(games_df, weekly_stats_w_rating)

X = test.drop(['DayNum','team_1_won'], axis=1)
y = test['team_1_won']

models = {
    'Decision Tree': DecisionTreeClassifier(random_state=3270, max_depth=10),
    'Random Forest': RandomForestClassifier(random_state=3270, n_estimators=200, max_depth=10, min_samples_split=10),
    'Logistic Regression': LogisticRegression(random_state=3270, max_iter=1000, penalty = None, solver = 'lbfgs', ),
    'XGBoost': XGBClassifier(random_state = 3270, n_estimators = 100, max_depth = 3, learning_rate = 0.1, gamma = 0, subsample = 0.8, colsample_bytree = 0.8)
}

for name, model in models.items():
    fit_model_scalar(model, name, X, y)
    fit_model(model, name, X, y)

Decision Tree scalar accuracy: 0.65
Decision Tree accuracy: 0.65
Random Forest scalar accuracy: 0.70
Random Forest accuracy: 0.70
Logistic Regression scalar accuracy: 0.71


c:\Users\Dowee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression accuracy: 0.72
XGBoost scalar accuracy: 0.71
XGBoost accuracy: 0.71


In [17]:
lgr = LogisticRegression(random_state=3270, max_iter=1000, penalty = None, solver = 'lbfgs')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3270)

lgr.fit(X_train, y_train)
y_pred = lgr.predict(X_test)
#Show the predictions and the actual values side by side
pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}).tail(50)
#Show the accuracy of the predictions for the last 50 games
accuracy_score(y_test[len(y_test)-200:], y_pred[len(y_test)-200:])
accuracy_score(y_test, y_pred)



c:\Users\Dowee\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7217659137577002

In [47]:
seasons_matchup_avgs_10_data = []
for season in range(2003 , 2025):
    season_games = pd.read_csv(f'{season}/MRegularSeasonDetailedResults_{season}_matchups_avg.csv')
    seasons_matchup_avgs_10_data.append(season_games)
    

In [145]:
def TimeSeriesSplit_by_season(seasons_data):
    """
    Splits the data into training and testing sets by season. Model is trained on all data up to a certain season and tested on the next season until the last season. 

    Parameters:
    - seasons_data (list): A list of DataFrames containing data for each season.

    Returns:
    - list: A list of tuples containing training and testing sets for each season.
    """
    scaler = StandardScaler()
    model = LogisticRegression(random_state=3270, max_iter=1000, penalty = None, solver = 'lbfgs')
    accuracies = []
    for i in range(1, len(seasons_data)):
        print(f'Testing on Season {seasons_data[i]["Season"].unique()[0]}')
        train = pd.concat(seasons_data[:i])
        train = train.dropna(axis = 'columns', how= 'any')
        X_train = train.drop(['Season','DayNum', 'team_1_won'], axis=1)
        y_train = train['team_1_won']
        X_train = scaler.fit_transform(X_train)
        test = seasons_data[i]
        test = test.dropna(axis = 'columns', how= 'any')
        X_test = test.drop(['Season','DayNum', 'team_1_won'], axis=1)
        X_test = scaler.transform(X_test)
        y_test = test['team_1_won']
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
        print(f'accuracy: {accuracy:.5f}')

    print(f'Average accuracy: {np.mean(accuracies):.5f}')
    return model
    

In [146]:
model = TimeSeriesSplit_by_season(seasons_matchup_avgs_10_data)





Testing on Season 2004
accuracy: 0.73660
Testing on Season 2005
accuracy: 0.72171
Testing on Season 2006
accuracy: 0.73345
Testing on Season 2007
accuracy: 0.73290
Testing on Season 2008
accuracy: 0.73233
Testing on Season 2009
accuracy: 0.73100
Testing on Season 2010
accuracy: 0.74045
Testing on Season 2011
accuracy: 0.74133
Testing on Season 2012
accuracy: 0.74700
Testing on Season 2013
accuracy: 0.72820
Testing on Season 2014
accuracy: 0.72902
Testing on Season 2015
accuracy: 0.72656
Testing on Season 2016
accuracy: 0.72993
Testing on Season 2017
accuracy: 0.72938
Testing on Season 2018
accuracy: 0.72710
Testing on Season 2019
accuracy: 0.72835
Testing on Season 2020
accuracy: 0.72616
Testing on Season 2021
accuracy: 0.71984
Testing on Season 2022
accuracy: 0.73658
Testing on Season 2023
accuracy: 0.71939
Testing on Season 2024
accuracy: 0.72074
Average accuracy: 0.73038


In [132]:
seasons_tourney_data = []
for season in range(2003 , 2025):
    season_games = pd.read_csv(f'{season}/MRegularSeasonCompactResults_{season}.csv')
    seasons_tourney_data.append(season_games)
    

In [133]:
team_stats_season = {}
for season in range(2003 , 2025):
    season_games = pd.read_csv(f'{season}/MRegularSeasonDetailedResults_{season}_avg.csv')
    team_stats_season[season] = season_games

In [134]:
def build_game_matchups(bracket_team_matchups, team_stats):
    """
    Builds matchup data for the tournament bracket.

    Parameters:
    - bracket_team_matchups (DataFrame): The DataFrame containing the current matchups for a bracket.

    Returns:
    - DataFrame: A DataFrame where each row contains team_1's and team_2's stats for that specific matchup.
    """
    matchups = []
    for _, row in bracket_team_matchups.iterrows():
        team_1 = row['StrongSeed']
        team_2 = row['WeakSeed']

        team_1_data = team_stats[team_stats['TeamID'] == team_1]
        team_2_data = team_stats[team_stats['TeamID'] == team_2]
        
        team_1_data.columns = [f'team_1_{col}' for col in team_1_data.columns]
        team_2_data.columns = [f'team_2_{col}' for col in team_2_data.columns]
        
        team_1_data = team_1_data.reset_index(drop=True)
        team_2_data = team_2_data.reset_index(drop=True)
        matchup_data = pd.concat([team_1_data, team_2_data], axis=1)
        matchup_data['Slot'] = row['Slot']
        matchups.append(matchup_data)
    return pd.concat(matchups, axis=0)

In [135]:
def preprocess_playins(seeds_df):
    """
    Preprocess the play in teams by removing 'a' and 'b' designations and preparing strong and weak seeds.
    
    Parameters:
    - seeds_df (DataFrame): The DataFrame containing tournament seeds data, including 'Seed', 'TeamID' columns.
    
    Returns:
    - DataFrame: Processed DataFrame with 'StrongSeed' and 'WeakSeed' for play-in teams.
    """
    playin_teams = seeds_df[seeds_df['Seed'].str.contains('a') | seeds_df['Seed'].str.contains('b')].copy()
    playin_teams['Seed'] = playin_teams['Seed'].str.extract('([0-9A-Z]+)')
    playin_teams_match_df = playin_teams.groupby('Seed')['TeamID'].apply(list).reset_index()
    playin_teams_match_df['StrongSeed'] = playin_teams_match_df['TeamID'].apply(lambda x: x[0])
    playin_teams_match_df['WeakSeed'] = playin_teams_match_df['TeamID'].apply(lambda x: x[1])
    playin_teams_match_df.rename(columns={'Seed': 'Slot'}, inplace=True)
    return playin_teams_match_df.drop(columns='TeamID')


In [136]:
def predict_bracket_winners(bracket_matchups, model, scalar):
    """
    Predict the winners in the lower bracket using a pre-trained model and scaler.
    
    Parameters:
    - bracket_matchups (DataFrame): DataFrame of matchups in the lower bracket, excluding 'Seed' from scaling.
    - model (Model): Pre-trained prediction model.
    - scalar (Scaler): Pre-fitted scaler object for normalizing data.
    
    Returns:
    - DataFrame: Lower bracket DataFrame with an additional column 'team_1_won' indicating the predicted winner.
    """
    bracket_matchups = bracket_matchups.drop(columns= [])
    lower_bracket_scaled = scalar.fit_transform(bracket_matchups.drop(columns=['Slot']))
    bracket_matchups['team_1_won'] = model.predict(lower_bracket_scaled)
    return bracket_matchups


In [137]:
def update_seeds_with_winners(bracket, seeds_df):
    """
    Update the seeds DataFrame with the winners from the lower bracket predictions.
    
    Parameters:
    - bracket (DataFrame): The lower bracket DataFrame with predictions.
    - seeds_df (DataFrame): The original seeds DataFrame to be updated with current teams seedings.
    
    Returns:
    - DataFrame: Updated seeds DataFrame with winners.
    """
    bracket_winners = {}
    for _, row in bracket.iterrows():
        slot = row['Slot']
        if slot not in bracket_winners:
            bracket_winners[slot] = []
        bracket_winners[slot].append(row['team_1_TeamID'] if row['team_1_won'] == 1 else row['team_2_TeamID'])
    
    for curr_seed, team in bracket_winners.items():
        seeds_df.loc[len(seeds_df.index)] = [curr_seed, team[0]]
    seeds_df.sort_values(by='Seed', inplace=True)
    return seeds_df


In [138]:
def build_round_matchups(team_seeds, round_slots):
    """
    Build the Tourney round matchups based on seeds and updates team slots.
    
    Parameters:
    - team_seeds (DataFrame): DataFrame containing the seeds and corresponding team IDs.
    - round_slots (DataFrame): DataFrame containing the slots for the tournament matchups.
    
    Returns:
    - DataFrame: First-round matchups with updated team slots based on seeds.
    """

    for index, row in round_slots.iterrows():
        strong_team = team_seeds[(team_seeds['Seed'] == row['StrongSeed'])]['TeamID'].values[0]
        weak_team = team_seeds[(team_seeds['Seed'] == row['WeakSeed'])]['TeamID'].values[0]
        round_slots.at[index, 'StrongSeed'] = strong_team
        round_slots.at[index, 'WeakSeed'] = weak_team

    return round_slots


In [139]:
tourney_seeds_df = pd.read_csv('2023/MNCAATourneySeeds_2023.csv')
tourney_seeds_df.drop(columns=['Season'], inplace=True)
tourney_slots_df = pd.read_csv('2023/MNCAATourneySlots_2023.csv')

scalar = StandardScaler()

playin_teams_match_df = preprocess_playins(tourney_seeds_df)
bracket_matchups = build_game_matchups(playin_teams_match_df, team_stats_season[2023].groupby('TeamID').last().reset_index())
bracket_matchups.columns
bracket_matchups = predict_bracket_winners(bracket_matchups, model, scalar)
tourney_seeds_df = update_seeds_with_winners(bracket_matchups, tourney_seeds_df)
print('Done with play-ins')


# curr_round_slots = tourney_slots_df[tourney_slots_df['Slot'].str.contains('R1')]

# round_df = build_round_matchups(tourney_seeds_df,curr_round_slots)

# current_round_bracket = build_game_matchups(round_df, team_stats_season[2023])
# current_round_bracket.reset_index(drop=True, inplace=True)
# current_round_bracket = predict_bracket_winners(current_round_bracket, model, scalar)
# tourney_seeds_df = update_seeds_with_winners(current_round_bracket, tourney_seeds_df)


rounds = ['R1', 'R2', 'R3', 'R4', 'R5', 'R6']
# rounds = ['R1']
matchups = []
for current_round in rounds:
    curr_round_slots = tourney_slots_df[tourney_slots_df['Slot'].str.contains(current_round)]

    round_matchups = build_round_matchups(tourney_seeds_df,curr_round_slots)
    matchups.append(round_matchups)
    current_round_bracket = build_game_matchups(round_matchups, team_stats_season[2023].groupby('TeamID').last().reset_index())
    current_round_bracket = predict_bracket_winners(current_round_bracket, model, scalar)
    tourney_seeds_df = update_seeds_with_winners(current_round_bracket, tourney_seeds_df)
    


Done with play-ins

In [144]:
# complete_bracket = pd.concat(matchups)
# complete_bracket.reset_index(drop=True, inplace=True)
# complete_bracket.to_csv('2023/MNCAATourneyPredictions_matchup_2023.csv', index=False)
# complete_bracket

# tourney_seeds_df
test_dict = {}
test_dict[type(lgr).__name__] = [accuracy_score(y_test, y_pred)]
test_dict

{'LogisticRegression': [0.7217659137577002]}